In [53]:
import numpy
import theano
import theano.tensor as T

import cPickle
import gzip
import os
import sys
import time

<h3>Logistic Regression</h3>
A logistic regression can be thought of as an artificial neural network with a single layer.  Given some data X and target classes y, a logistic regression is parameterized by a single weight matrix W and bias vector b.


The model gives the probability that some data x belongs to class i,

$$P(Y = i| x, W, b) = softmax_i(Wx + b)$$

Where the softmax function normalizes a series of responses to $[0, 1]$,

$$softmax_i = \frac{e^{W_ix + b_i}}{\sum_j e^{W_jx + b_j}} $$

And the predicted class is simply
$$y_{pred} = argmax_i P(Y = i |x, W, b) $$

In [18]:
class LogisticRegression(object):
    """Multicalss logistic regression object"""
    
    def __init__(self, input, n_in, n_out):
        self.W = theano.shared(value=numpy.zeros((n_in, n_out),
                                      dtype=theano.config.floatX),
                               name='W', borrow=True)
        self.b = theano.shared(value=numpy.zeros((n_out,),
                                      dtype=theano.config.floatX),
                               name='b', borrow=True)

        self.p_y_given_x = T.nnet.softmax(T.dot(input, self.W) + self.b)
        self.y_pred = T.argmax(self.p_y_given_x, axis=1)
        self.params = [self.W, self.b]
        
    def negative_log_likelihood(self, y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
    
    def errors(self, y):
        """Return a float representing the number of errors in the minibatch
        over the total number of examples of the minibatch ; zero one
        loss over the size of the minibatch

        :type y: theano.tensor.TensorType
        :param y: corresponds to a vector that gives for each example the
                  correct label
        """

        # check if y has same dimension of y_pred
        if y.ndim != self.y_pred.ndim:
            raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', self.y_pred.type)
            )
        # check if y is of the correct datatype
        if y.dtype.startswith('int'):
            # the T.neq operator returns a vector of 0s and 1s, where 1
            # represents a mistake in prediction
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()
        

In [42]:
from sklearn.datasets import load_digits
x = load_digits()['data']
y = load_digits()['target']

In [51]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y)

x_train = theano.shared(numpy.asarray(x_train, dtype=theano.config.floatX), borrow=True)
y_train = theano.shared(numpy.asarray(y_train, dtype=theano.config.floatX), borrow=True)
y_train = T.cast(y_train, 'int32')

x_test = theano.shared(numpy.asarray(x_test, dtype=theano.config.floatX), borrow=True)
y_test = theano.shared(numpy.asarray(y_test, dtype=theano.config.floatX), borrow=True)
y_test = T.cast(y_test, 'int32')

In [34]:
x_train.shape

(1347, 64)

In [37]:
x = T.matrix('x')
y = T.ivector('y')
classifier = LogisticRegression(input=x, n_in=64, n_out=10)
cost = classifier.negative_log_likelihood(y)

g_W = T.grad(cost=cost, wrt=classifier.W)
g_b = T.grad(cost=cost, wrt=classifier.b)

In [56]:
def sgd_optimization_mnist(learning_rate=0.13, n_epochs=1000,
                           batch_size=10):
    """
    Demonstrate stochastic gradient descent optimization of a log-linear
    model

    This is demonstrated on MNIST.

    :type learning_rate: float
    :param learning_rate: learning rate used (factor for the stochastic
                          gradient)

    :type n_epochs: int
    :param n_epochs: maximal number of epochs to run the optimizer

    :type dataset: string
    :param dataset: the path of the MNIST dataset file from
                 http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz

    """

    train_set_x, train_set_y = (x_train, y_train)
    valid_set_x, valid_set_y = (x_test, y_test)
    test_set_x, test_set_y = (x_test, y_test)

    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size


 
    #----------- BUILD ACTUAL MODEL ----------------#
    print '... building the model'

    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch

    # generate symbolic variables for input (x and y represent a
    # minibatch)
    x = T.matrix('x')  # data, presented as rasterized images
    y = T.ivector('y')  # labels, presented as 1D vector of [int] labels


    classifier = LogisticRegression(input=x, n_in=64, n_out=10)
    cost = classifier.negative_log_likelihood(y)

    # compiling a Theano function that computes the mistakes that are made by
    # the model on a minibatch
    test_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    validate_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    # compute the gradient of cost with respect to theta = (W,b)
    g_W = T.grad(cost=cost, wrt=classifier.W)
    g_b = T.grad(cost=cost, wrt=classifier.b)

    # start-snippet-3
    # specify how to update the parameters of the model as a list of
    # (variable, update expression) pairs.
    updates = [(classifier.W, classifier.W - learning_rate * g_W),
               (classifier.b, classifier.b - learning_rate * g_b)]

    # compiling a Theano function `train_model` that returns the cost, but in
    # the same time updates the parameter of the model based on the rules
    # defined in `updates`
    train_model = theano.function(
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )
    # end-snippet-3

    ###############
    # TRAIN MODEL #
    ###############
    print '... training the model'
    # early-stopping parameters
    patience = 5000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                                  # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                  # considered significant
    validation_frequency = min(n_train_batches, patience / 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = numpy.inf
    test_score = 0.
    start_time = time.clock()

    done_looping = False
    epoch = 0
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in xrange(n_train_batches):

            minibatch_avg_cost = train_model(minibatch_index)
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:
                # compute zero-one loss on validation set
                validation_losses = [validate_model(i)
                                     for i in xrange(n_valid_batches)]
                this_validation_loss = numpy.mean(validation_losses)

                print(
                    'epoch %i, minibatch %i/%i, validation error %f %%' %
                    (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        this_validation_loss * 100.
                    )
                )

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if this_validation_loss < best_validation_loss *  \
                       improvement_threshold:
                        patience = max(patience, iter * patience_increase)

                    best_validation_loss = this_validation_loss
                    # test it on the test set

                    test_losses = [test_model(i)
                                   for i in xrange(n_test_batches)]
                    test_score = numpy.mean(test_losses)

                    print(
                        (
                            '     epoch %i, minibatch %i/%i, test error of'
                            ' best model %f %%'
                        ) %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            test_score * 100.
                        )
                    )

            if patience <= iter:
                done_looping = True
                break

    end_time = time.clock()
    print(
        (
            'Optimization complete with best validation score of %f %%,'
            'with test performance %f %%'
        )
        % (best_validation_loss * 100., test_score * 100.)
    )
    print 'The code run for %d epochs, with %f epochs/sec' % (
        epoch, 1. * epoch / (end_time - start_time))

if __name__ == '__main__':
    sgd_optimization_mnist()

... building the model
... training the model
epoch 1, minibatch 134/134, validation error 9.555556 %
     epoch 1, minibatch 134/134, test error of best model 9.555556 %
epoch 2, minibatch 134/134, validation error 15.111111 %
epoch 3, minibatch 134/134, validation error 5.333333 %
     epoch 3, minibatch 134/134, test error of best model 5.333333 %
epoch 4, minibatch 134/134, validation error 8.444444 %
epoch 5, minibatch 134/134, validation error 7.777778 %
epoch 6, minibatch 134/134, validation error 5.555556 %
epoch 7, minibatch 134/134, validation error 4.888889 %
     epoch 7, minibatch 134/134, test error of best model 4.888889 %
epoch 8, minibatch 134/134, validation error 5.777778 %
epoch 9, minibatch 134/134, validation error 5.777778 %
epoch 10, minibatch 134/134, validation error 7.111111 %
epoch 11, minibatch 134/134, validation error 4.666667 %
     epoch 11, minibatch 134/134, test error of best model 4.666667 %
epoch 12, minibatch 134/134, validation error 3.555556 %
 

In [49]:
x.shape

(1797, 64)